<a href="https://colab.research.google.com/github/TaruSora/NLP_100knock/blob/main/100knock_9_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
! pip install torch==1.6.0
! pip install torchtext==0.7.0
! pip install torch_optimizer
! pip install pytorch-lightning==1.0.8

     |████████████████████████████████| 748.8 MB 12 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.


     |████████████████████████████████| 4.5 MB 4.3 MB/s 
     |████████████████████████████████| 1.2 MB 52.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0
     |████████████████████████████████| 61 kB 358 kB/s 
     |████████████████████████████████| 561 kB 4.2 MB/s 
     |████████████████████████████████| 829 kB 49.9 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
     |████████████████████████████████| 136 kB 65.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=1afe17e754431153321f835200bc7d6e9e08a87d218c91cdd9b1c5b5bd6a4a45
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled futur

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
!unzip NewsAggregatorDataset.zip
!sed -e 's/"/'\''/g' newsCorpora.csv > newsCorpora_re.csv

--2022-04-01 05:04:07--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  13.2MB/s    in 2.1s    

2022-04-01 05:04:10 (13.2 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [3]:
# 50のコードをそのまま利用

import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('newsCorpora_re.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]

train, tmp = train_test_split(df, test_size=0.2, shuffle=True, random_state=42, stratify=df['CATEGORY'])
valid, test = train_test_split(tmp, test_size=0.5, shuffle=True, random_state=42, stratify=tmp['CATEGORY'])

train.to_csv('train.txt', sep='\t', index=False)
valid.to_csv('valid.txt', sep='\t', index=False)
test.to_csv('test.txt', sep='\t', index=False)

print('学習用データ')
print(train['CATEGORY'].value_counts())
print('評価用データ')
print(valid['CATEGORY'].value_counts())
print('テスト用データ')
print(test['CATEGORY'].value_counts())

学習用データ
b    4501
e    4235
t    1220
m     728
Name: CATEGORY, dtype: int64
評価用データ
b    563
e    529
t    153
m     91
Name: CATEGORY, dtype: int64
テスト用データ
b    563
e    530
t    152
m     91
Name: CATEGORY, dtype: int64


In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
import torch_optimizer as optim
from torch.utils.data import DataLoader
from torchtext.data import Example, Field, Dataset, BucketIterator
from torchtext.vocab import FastText
import string
import os

# データ読み込み
text_field = Field(sequential=True, use_vocab=True)
label_field = Field(sequential=False, use_vocab=False, is_target=True)
fields = [("x", text_field), ("t", label_field)]

table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
label2id = {'b': 0, 't': 1, 'e': 2, 'm': 3}
batch_size = 1

def load_corpus(fname):
    examples = list()
    with open(fname, 'r') as f:
        sentences = df["TITLE"]
        labels = df["CATEGORY"]
        for sentence, label in zip(sentences, labels):
            word_list = sentence.translate(table).split()
            label_id = label2id[label]
            examples.append(Example.fromlist([word_list, label_id], fields))
        return Dataset(examples, fields)

train_dataset = load_corpus('train.txt')
valid_dataset = load_corpus('valid.txt')
test_dataset = load_corpus('test.txt')

text_field.build_vocab(train_dataset, vectors=FastText(language="en"), min_freq=2)

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
.vector_cache/wiki.en.vec:  18%|█▊        | 1.17G/6.60G [3:02:30<15:26:06, 97.7kB/s]

In [12]:
class MyRNN(pl.LightningModule):
    # 層の定義
    def __init__(self, n_input, n_emb, n_hidden, n_layers, n_output, dropout, bidirectional, lr):
        super(MyRNN, self).__init__()
        self.embed = nn.Embedding(num_embeddings=n_input, embedding_dim=n_emb, padding_idx=1)
        self.embed.weight.data.copy_(text_field.vocab.vectors)
        self.lstm = nn.LSTM(input_size=n_emb, hidden_size=n_hidden, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        self.fc = nn.Linear(in_features=n_hidden * (2 if bidirectional==True else 1), out_features=n_output)    
    # 順伝播
    def forward(self, x):
        o, (h, c) = self.lstm(self.embed(x))
        return self.fc(o[-1])
    # 訓練用データの損失計算
    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("train_loss", loss)
        return loss
    # 検証用データの損失計算
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("val_loss", loss)
    # 評価用データの正解率計算
    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x) 
        y = torch.argmax(y, dim=1)
        accuracy = torch.sum(t == y).item() / (len(y) * 1.0)
        self.log("test_acc", accuracy)
    # 損失関数の定義
    def lossfun(self, y, t):
        return F.cross_entropy(y, t)
    # オプティマイザの定義
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1)

88.問題85や問題87のコードを改変し，ニューラルネットワークの形状やハイパーパラメータを調整しながら，高性能なカテゴリ分類器を構築せよ．

In [ ]:
!pip install optuna

In [ ]:
# 88 パラメータチューニング

import optuna

# optunaによるハイパーパラメータチューニング
def objective(trial):
    n_hidden = int(trial.suggest_discrete_uniform('n_hidden', 100, 200, 50))
    n_layers = int(trial.suggest_discrete_uniform('n_layers', 1, 3, 1))
    batch_size = int(trial.suggest_discrete_uniform('batch_size', 32, 64, 32))

    n_input = len(text_field.vocab)
    n_embed = 300
    n_output = len(label2id)
    bidirectional = True
    dropout = 0.3
    lr = 0.01

    train_dataloader = BucketIterator(train_dataset, batch_size=batch_size, shuffle=True)
    valid_dataloader = BucketIterator(valid_dataset, batch_size=batch_size, shuffle=False)
    test_dataloader = BucketIterator(test_dataset, batch_size=batch_size, shuffle=False)


    model = MyRNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional, lr)

    # 訓練中にモデルを保存するための設定
    checkpoint = pl.callbacks.ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1,save_weights_only=True, dirpath="model/")
 
    # 訓練
    trainer = pl.Trainer(gpus=1, max_epochs=20, callbacks=[checkpoint])
    trainer.fit(model, train_dataloader, valid_dataloader)

    valid_loss = checkpoint.best_model_score

    return valid_loss

In [ ]:
print('Best trial:')
trial = study.best_trial
print('  Value: {:.3f}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
  print('    {}: {}'.format(key, value))

In [ ]:
batch_size = trial.params['batch_size']

train_dataloader = BucketIterator(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = BucketIterator(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = BucketIterator(test_dataset, batch_size=batch_size, shuffle=False)

n_input = len(text_field.vocab)
n_embed = 300
n_hidden = int(trial.params['n_hidden'])
n_layers = int(trial.params['n_layers'])
n_output = len(label2id)
dropout = 0.3
lr = 0.01
bidirectional = True

model = MyRNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional, lr)

# 訓練中にモデルを保存するための設定
checkpoint = pl.callbacks.ModelCheckpoint(
    # 検証用データにおける損失が最も小さいモデルを保存する
    monitor="val_loss", mode="min", save_top_k=1,
    # モデルファイル（重みのみ）を "model" というディレクトリに保存する
    save_weights_only=True, dirpath="model/"
)

# 訓練
trainer = pl.Trainer(gpus=1, max_epochs=10, callbacks=[checkpoint])
trainer.fit(model, train_dataloader, valid_dataloader)

# ベストモデルの確認
print("ベストモデル: ", checkpoint.best_model_path)
print("ベストモデルの検証用データにおける損失: ", checkpoint.best_model_score)

# 評価
test = trainer.test(test_dataloaders=test_dataloader)
print("Test accuracy = %.3f" % (test[0]["test_acc"]))

89.事前学習済み言語モデル（例えばBERTなど）を出発点として，ニュース記事見出しをカテゴリに分類するモデルを構築せよ．

In [ ]:
import time

def calc_loss_acc(model, device, dataloader, criterion):
    loss = 0.0
    total = 0
    correct = 0
    with torch.no_grad():
        for data in dataloader:
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            labels = data['labels'].to(device)
            y = model(ids, mask)

            if criterion != None:
                loss += criterion(y, labels).item()

            pred = torch.argmax(y, dim=-1).cpu().numpy()
            labels = torch.argmax(labels, dim=-1).cpu().numpy()
            total += len(labels)
            correct += (pred == labels).sum().item()

    return loss / len(dataloader), correct / total

def model_function(train_dataset, valid_dataset, batch_size, device, model, criterion, optimizer, max_epoch):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=len(valid_dataset), shuffle=False)

    train_log = []
    valid_log = []
    
    for epoch in range(max_epoch):
        start = time.time()

        model.train()
        for data in train_loader:
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            labels = data['labels'].to(device)

            optimizer.zero_grad()

            y = model(ids, mask)
            loss = criterion(y, labels)        
            loss.backward()
            optimizer.step()

        model.eval()

        train_loss, train_acc = calc_loss_acc(model, device, train_loader, criterion)
        valid_loss, valid_acc = calc_loss_acc(model, device, valid_loader, criterion)
        train_log.append([train_loss, train_acc])
        valid_log.append([valid_loss, valid_acc])   

        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, f'checkpoint{epoch + 1}.pt')

        end = time.time() 

        print(f'epoch: {epoch + 1}, loss_train: {train_loss:.4f}, accuracy_train: {train_acc:.4f}, loss_valid: {valid_loss:.4f}, accuracy_valid: {valid_acc:.4f}, {(end - start):.4f}sec') 

    return {'train': train_log, 'valid': valid_log}

In [ ]:
import torch.nn as nn

class BertClass(torch.nn.Module):
    def __init__(self, dropout, output_size):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(dropout)
        self.fc = torch.nn.Linear(768, output_size)

    def forward(self, ids, mask):
        _, output = self.bert(ids, attention_mask=mask, return_dict=False)
        output = self.fc(self.dropout(output))
        return output    

dropout = 0.3
output_size = 4
batch_size = 16
max_epochs = 5
lr = 2e-5

model = BertClass(dropout, output_size)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model_function(train_dataset, valid_dataset, batch_size, device, model, criterion, optimizer, max_epochs)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=len(valid_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
_, train_acc = loss_and_accuracy(model, device, train_loader, criterion)
_, valid_acc = loss_and_accuracy(model, device, valid_loader, criterion)
_, test_acc = loss_and_accuracy(model, device, test_loader, criterion)
print(f'正解率（学習データ）：{train_acc:.3f}')
print(f'正解率（学習データ）：{valid_acc:.3f}')
print(f'正解率（評価データ）：{test_acc:.3f}')